In [1]:
import os 

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Bls17/datascience-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Bls17"
os.environ["MLFLOW_TRACKING_PASSWORD"]="61087f41394d9ae01fd32f4ba34d8d0bd3cd9f48"

In [3]:
os.chdir("../")
%pwd

'/Users/badaradiallo/Desktop/DataScience_Projetc'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path:Path
    model_path:Path
    all_params:dict
    metric_file_name:str
    target_column:str
    mlflow_uri:str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH
                 ,schema_filepath = SCHEMA_FILE_PATH): 
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config= ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Bls17/datascience-project.mlflow"
        )
        
        return model_evaluation_config

In [7]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self,config = ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mse = mean_squared_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse,mse,r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            # Model registry does not work with file store or unsupported backends (like DagsHub)
            if tracking_url_type_store != "file" and "dagshub" not in self.config.mlflow_uri:
                mlflow.sklearn.log_model(model,name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e: 
    raise e

[2025-06-23 15:45:49,389: INFO: common: yaml file: /Users/badaradiallo/Desktop/DataScience_Projetc/config/config.yaml loaded successfully]
[2025-06-23 15:45:49,394: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-23 15:45:49,395: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-23 15:45:49,397: INFO: common: created directory at: artifacts]
[2025-06-23 15:45:49,398: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-23 15:45:49,649: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
